# 1. Imports

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive/', force_remount=True)

# %cd gdrive/MyDrive/BT4222 Project Group/Codes

Mounted at /content/gdrive/


In [ ]:
!pip install keras-tuner
!pip install optuna

shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
The folder you are executing pip from can no longer be found.
shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
The folder you are executing pip from can no longer be found.


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras import layers
import tensorflow as tf
from keras.utils.vis_utils import plot_model
from keras_tuner import RandomSearch
from keras.models import save_model, load_model
import random
import warnings
warnings.filterwarnings("ignore")
from keras import backend as K
import optuna
from time import time
from sklearn.metrics import f1_score, accuracy_score
import pickle

# 2. Read in datasets

In [ ]:
rdn_index = random.sample(range(110248), 60000)

In [ ]:
# word2vec: skip-gram
X_train = pd.read_csv('../Word Embedding/emb_sg_train.csv').iloc[rdn_index]
X_test = pd.read_csv('../Word Embedding/emb_sg_test.csv')

# labels
y_train = pd.read_csv('../Data/y_train.csv')['class'].iloc[rdn_index]
y_test = pd.read_csv('../Data/y_test.csv')['class']

In [ ]:
embedding_length = X_train.shape[1]
input_dimension = X_train.shape[0]

print(f'Embedding Length: {embedding_length} \nInput Dimension: {input_dimension}')

Embedding Length: 100 
Input Dimension: 110248


# 3. Model Tuning

3a. CNN

In [ ]:
def build_model_cnn(hp):

    # create model object
    model = keras.Sequential([
      layers.Input(shape=(embedding_length,1)),
      layers.Conv1D(
          filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16), # number of filters
          kernel_size=hp.Choice('conv_1_kernel', values = [3,5,7]), # filter size
          activation='relu',
      ),
      layers.MaxPooling1D(),
      layers.Conv1D(
          filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16), # number of filters
          kernel_size=hp.Choice('conv_2_kernel', values = [3,5,7]), # filter size
          activation='relu',
      ),
      layers.GlobalMaxPooling1D(), 
      layers.Dense(
          units=hp.Int('dense_1_units', min_value=32, max_value=256, step=16),
          activation='relu'
      ),
      layers.Dropout(
          hp.Choice('dropout_1', values = [0.1, 0.2, 0.3])
      ),
      layers.Dense(
          units=hp.Int('dense_2_units', min_value=32, max_value=256, step=16),
          activation='relu'
      ),
      layers.Dropout(
          hp.Choice('dropout_2', values = [0.1, 0.2, 0.3, 0.4])
      ),
      # output layer    
      layers.Dense(1, activation='sigmoid', name = 'Output') # output
    ])
    #compilation of model
    model.compile(
        optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
        loss='binary_crossentropy',
        metrics=['accuracy']
        )
    # return model
    return model

In [ ]:
#creating randomsearch object
cnn_tuner = RandomSearch(build_model_cnn, objective='val_accuracy', max_trials = 5)

# search best parameter
cnn_tuner.search(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

In [ ]:
cnn = cnn_tuner.get_best_models(num_models=1)[0]

#summary of best model
cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 96, 64)            384       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 48, 64)           0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 46, 48)            9264      
                                                                 
 global_max_pooling1d (Globa  (None, 48)               0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 128)               6272      
                                                                 
 dropout (Dropout)           (None, 128)               0

In [ ]:
# Fit model
history = cnn.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size = 200)

Epoch 1/10
552/552 [==============================] - 28s 48ms/step - loss: 0.2883 - accuracy: 0.8858 - val_loss: 0.2444 - val_accuracy: 0.8993
Epoch 2/10
552/552 [==============================] - 26s 47ms/step - loss: 0.2306 - accuracy: 0.9094 - val_loss: 0.2225 - val_accuracy: 0.9129
Epoch 3/10
552/552 [==============================] - 26s 47ms/step - loss: 0.2269 - accuracy: 0.9114 - val_loss: 0.2222 - val_accuracy: 0.9128
Epoch 4/10
552/552 [==============================] - 25s 46ms/step - loss: 0.2219 - accuracy: 0.9131 - val_loss: 0.2193 - val_accuracy: 0.9147
Epoch 5/10
552/552 [==============================] - 27s 48ms/step - loss: 0.2406 - accuracy: 0.9075 - val_loss: 0.2503 - val_accuracy: 0.9038
Epoch 6/10
552/552 [==============================] - 25s 45ms/step - loss: 0.2365 - accuracy: 0.9076 - val_loss: 0.2275 - val_accuracy: 0.9117
Epoch 7/10
552/552 [==============================] - 24s 44ms/step - loss: 0.2226 - accuracy: 0.9133 - val_loss: 0.2264 - val_accuracy:

In [ ]:
# Save model
save_model(cnn, '../Models/CNN.h5')

3b. LSTM

In [ ]:
lstm_num_neurons = [64, 128]
lstm_dropout = [0.1, 0.2]
lstm_rec_dropout = [0.1, 0.2]
dense_1_dropout = [0.1, 0.2, 0.3]

eval_table = pd.DataFrame()

for lstm_num_neurons_val in lstm_num_neurons:
  for lstm_dropout_val in lstm_dropout:
    for lstm_rec_dropout_val in lstm_rec_dropout:
      for dense_1_dropout_val in dense_1_dropout:

        temp_model = keras.Sequential([
          layers.Input(shape=(X_train.shape[1],1)),
          layers.LSTM(
              lstm_num_neurons_val, 
              dropout=lstm_dropout_val, 
              recurrent_dropout=lstm_rec_dropout_val, 
              return_sequences=True
              ),
          layers.GlobalMaxPooling1D(),
          layers.Dense(256, activation='relu'),
          layers.Dropout(dense_1_dropout_val),
          layers.Dense(128, activation='relu'),
          layers.Dense(1, activation='sigmoid')
        ])

        temp_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        temp_model.fit(X_train, y_train, batch_size=1000, epochs=5)

        y_prob = temp_model.predict(X_test).reshape(-1,1)
        y_pred = list(map(lambda x: 1 if x > 0.5 else 0, y_prob))
        eval_table = eval_table.append({
            'Params': {
                'LSTM_Number of neurons': lstm_num_neurons_val,
                'LSTM_Dropout': lstm_dropout_val,
                'LSTM_Rec_Dropout': lstm_rec_dropout_val,
                'Dense_Dropout': dense_1_dropout_val
            },
            'Accuracy': accuracy_score(y_pred, y_test),
            'Recall': recall_score(y_pred, y_test),
            'Precision': precision_score(y_pred, y_test),
            'F1-Score': f1_score(y_pred, y_test)
        }, ignore_index = True)

Epoch 1/5
111/111 [==============================] - 111s 973ms/step - loss: 0.5318 - accuracy: 0.7412
Epoch 2/5
111/111 [==============================] - 107s 963ms/step - loss: 0.4166 - accuracy: 0.8160
Epoch 3/5
111/111 [==============================] - 107s 964ms/step - loss: 0.3720 - accuracy: 0.8387
Epoch 4/5
111/111 [==============================] - 110s 990ms/step - loss: 0.3492 - accuracy: 0.8519
Epoch 5/5
862/862 [==============================] - 15s 18ms/step
Epoch 1/5
111/111 [==============================] - 111s 977ms/step - loss: 0.5389 - accuracy: 0.7380
Epoch 2/5
111/111 [==============================] - 107s 965ms/step - loss: 0.4198 - accuracy: 0.8141
Epoch 3/5
111/111 [==============================] - 111s 997ms/step - loss: 0.3665 - accuracy: 0.8420
Epoch 4/5
111/111 [==============================] - 107s 966ms/step - loss: 0.3348 - accuracy: 0.8584
Epoch 5/5
862/862 [==============================] - 18s 20ms/step
Epoch 1/5
111/111 [=======================

In [ ]:
# Get the top F1-Score + top Accuracy score
subset = eval_table[eval_table['F1-Score'] == eval_table['F1-Score'].max()]
subset = subset[subset['Accuracy'] == subset['Accuracy'].max()].iloc[0,:]
print(subset)
params = subset['Params']

# Build final model
final_lstm_model = keras.Sequential([
          layers.Input(shape=(X_train.shape[1],1)),
          layers.LSTM(
              params['LSTM_Number of neurons'], 
              dropout=params['LSTM_Dropout'], 
              recurrent_dropout=params['LSTM_Rec_Dropout'], 
              return_sequences=True
              ),
          layers.GlobalMaxPooling1D(),
          layers.Dense(256, activation='relu'),
          layers.Dropout(params['Dense_Dropout']),
          layers.Dense(256, activation='relu'),
          layers.Dense(1, activation='sigmoid')
        ])

final_lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
final_lstm_model.fit(X_train, y_train, batch_size=1000, epochs=10)

# save model
# codes to predict
# y_prob = temp_model.predict(X_test).reshape(-1,1)
# y_pred = list(map(lambda x: 1 if x > 0.5 else 0, y_prob))

Params       {'LSTM_Number of neurons': 128, 'LSTM_Dropout'...
Accuracy                                              0.894892
Recall                                                0.857736
Precision                                             0.877796
F1-Score                                               0.86765
Name: 16, dtype: object
Epoch 1/10
111/111 [==============================] - 234s 2s/step - loss: 0.5171 - accuracy: 0.7519
Epoch 2/10
111/111 [==============================] - 230s 2s/step - loss: 0.3917 - accuracy: 0.8274
Epoch 3/10
111/111 [==============================] - 226s 2s/step - loss: 0.3464 - accuracy: 0.8521
Epoch 4/10
111/111 [==============================] - 227s 2s/step - loss: 0.3202 - accuracy: 0.8649
Epoch 5/10
111/111 [==============================] - 224s 2s/step - loss: 0.3073 - accuracy: 0.8719
Epoch 6/10
111/111 [==============================] - 229s 2s/step - loss: 0.2958 - accuracy: 0.8779
Epoch 7/10
111/111 [==============================] - 2

In [ ]:
# Save model
save_model(final_lstm_model, '../Models/LSTM.h5')

3c. XGBoost

Maximize for F1 Score.

***Params***:
- **max_depth**: control max depth of tree, control overfitting
- **learning_rate**: to prevent overfitting
- **gamma**: specifies the minimum loss reduction required to make a node split, makes algorithm conservative
- **reg_lambda**: L2 regularization on weights, makes model conservative
- **subsample**: use random subset of data -> speed up cross validation + prevent overfitting
- **colsample_bytree**: use random subset of features -> speed up cross validation + prevent overfitting

In [ ]:
# surpress optuna trial message
optuna.logging.set_verbosity(optuna.logging.WARNING)

In [ ]:
import xgboost as xgb

In [ ]:
# Run this code to tune our hyperparameters via optuna
def objective(trial):

    params = {
        'objective': 'binary:logistic',
        'max_depth': trial.suggest_int("max_depth", 3, 10), 
        'learning_rate': trial.suggest_float("learning_rate", 0.01, 0.2),
        'gamma': trial.suggest_float("gamma", 0, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 10.0),
        'subsample': trial.suggest_float("subsample", 0.7, 1.0),
        'colsample_bytree': trial.suggest_float("colsample_bytree", 0.5, 1.0),
        'n_estimators': 10                  
    }

    model = xgb.XGBClassifier(**params)

    model.fit(X_train,
              y_train,
              verbose=0,
              early_stopping_rounds=10,
              eval_metric='auc',                # use roc_auc because of balanced data
              eval_set=[(X_test, y_test)])
  
    prediction = model.predict(X_test)
    f1 = f1_score(y_test, prediction)
    return f1

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)         # change n_trials as see fit

print("Best trial:")
trial = study.best_trial
print("  F1-Value: ",  trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print(".   {}: {}".format(key, value))

Best trial:
  F1-Value:  0.866400937866354
  Params: 
.   max_depth: 10
.   learning_rate: 0.17733982601791481
.   gamma: 0.34128807644815795
.   reg_lambda: 1.0027519059241101
.   subsample: 0.90892515690997
.   colsample_bytree: 0.5878770839064508


In [ ]:
# save model
file_name = "../Models/xgb.pkl"
xgb_model = xgb.XGBClassifier(**trial.params) 
xgb_model.fit(X_train, y_train, verbose=0, early_stopping_rounds=10, eval_metric='auc', eval_set=[(X_test, y_test)]) # use roc_auc because of balanced data
pickle.dump(xgb_model, open(file_name, "wb"))

# load
xgb_model_loaded = pickle.load(open(file_name, "rb"))

3d. LightGBM

In [ ]:
import lightgbm as lgb
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Change n_estimators and n_trials as see fit as it affects runtime!!!!

def objective(trial):

    params = {
        'objective': 'binary',
        'boosting': 'gbdt',
        'metric': 'AUC',
        'num_boost_round': 10,                       # change
        'num_leaves': trial.suggest_int("num_leaves", 2, 100),  
        'max_depth': trial.suggest_int("max_depth", 3, 10), 
        'learning_rate': trial.suggest_float("learning_rate", 0.01, 0.2),        
        'feature_fraction': trial.suggest_float('feature_fraction', 0.7, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
    }

    
    model = lgb.LGBMClassifier(**params)

    model.fit(X_train,
              y_train,
              verbose=False,
              early_stopping_rounds=10,
              eval_metric='auc',                # use roc_auc because of balanced data
              eval_set=[(X_test, y_test)])
  
    prediction = model.predict(X_test)
    f1 = f1_score(y_test, prediction)
    return f1

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)         # change n_trials as see fit

print("Best trial:")
trial = study.best_trial
print("  F1-Value: ",  trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print(".   {}: {}".format(key, value))

Best trial:
  F1-Value:  0.8455836641002343
  Params: 
.   num_leaves: 28
.   max_depth: 6
.   learning_rate: 0.19812089445259645
.   feature_fraction: 0.782974065106526
.   bagging_fraction: 0.6758979087732706
.   bagging_freq: 6


In [ ]:
# save model
file_name = "../Models/lgb.pkl"
lgb_model = lgb.LGBMClassifier(**trial.params) 
lgb_model.fit(X_train, y_train, verbose=0, early_stopping_rounds=10, eval_metric='auc', eval_set=[(X_test, y_test)]) # use roc_auc because of balanced data
pickle.dump(lgb_model, open(file_name, "wb"))

# load
lgb_model_loaded = pickle.load(open(file_name, "rb"))

3e. SVM

In [ ]:
from sklearn.svm import SVC

In [ ]:
def objective(trial):

    params = {
        'kernel': trial.suggest_categorical('kernel', ['linear', 'poly', 'rbf', 'sigmoid']),
        'C': trial.suggest_float('C', 0.01, 10),
        'degree': trial.suggest_int('degree', 1, 5, 1),
    }

    model = SVC(**params)

    model.fit(X_train, y_train)
  
    prediction = model.predict(X_test)
    f1 = f1_score(y_test, prediction)
    return f1

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=25)         # change n_trials as see fit

print("Best trial:")
trial = study.best_trial
print("  F1-Value: ",  trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print(".   {}: {}".format(key, value))

[I 2022-11-09 09:15:51,539] A new study created in memory with name: no-name-37d21983-ae16-44c4-be78-52fd78aee0ec
[I 2022-11-09 09:15:54,870] Trial 0 finished with value: 0.8730464981670846 and parameters: {'kernel': 'poly', 'C': 5.502704621678023, 'degree': 1}. Best is trial 0 with value: 0.8730464981670846.
[I 2022-11-09 09:16:00,502] Trial 1 finished with value: 0.5792072224573284 and parameters: {'kernel': 'sigmoid', 'C': 0.7056257986926229, 'degree': 3}. Best is trial 0 with value: 0.8730464981670846.
[I 2022-11-09 09:16:05,894] Trial 2 finished with value: 0.5772012505249404 and parameters: {'kernel': 'sigmoid', 'C': 3.034922503600857, 'degree': 2}. Best is trial 0 with value: 0.8730464981670846.
[I 2022-11-09 09:16:11,855] Trial 3 finished with value: 0.869195457840058 and parameters: {'kernel': 'rbf', 'C': 1.6377790557557914, 'degree': 3}. Best is trial 0 with value: 0.8730464981670846.
[I 2022-11-09 09:16:17,923] Trial 4 finished with value: 0.7039935550696284 and parameters: 

Best trial:
  F1-Value:  0.8832559461764427
  Params: 
.   kernel: rbf
.   C: 9.804933657998198
.   degree: 3


In [ ]:
# save model
file_name = "../Models/svm.pkl"
svm_model = SVC(**trial.params) 
svm_model.fit(X_train, y_train)
pickle.dump(svm_model, open(file_name, "wb"))

# load
svm_model_loaded = pickle.load(open(file_name, "rb"))

3f. Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
def create_model(trial):

    regularization = trial.suggest_float('C', 0.01, 100)
    model = LogisticRegression(C=regularization, max_iter=150)
    if trial.should_prune():
            raise optuna.TrialPruned()
            
    return model


def objective(trial):
    model = create_model(trial)
    model.fit(X_train, y_train)

    prediction = model.predict(X_test)
    f1 = f1_score(y_test, prediction)
    return f1

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=25)         # change n_trials as see fit

print("Best trial:")
trial = study.best_trial
print("  F1-Value: ",  trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print(".   {}: {}".format(key, value))

[I 2022-11-09 08:06:42,093] A new study created in memory with name: no-name-869c99ff-9991-4693-910a-752a2ecc57a9
[I 2022-11-09 08:06:43,631] Trial 0 finished with value: 0.8823137665303015 and parameters: {'C': 78.11692480616955}. Best is trial 0 with value: 0.8823137665303015.
[I 2022-11-09 08:06:46,208] Trial 1 finished with value: 0.8823137665303015 and parameters: {'C': 18.017806946549708}. Best is trial 0 with value: 0.8823137665303015.
[I 2022-11-09 08:06:48,681] Trial 2 finished with value: 0.8823137665303015 and parameters: {'C': 18.184758814332202}. Best is trial 0 with value: 0.8823137665303015.
[I 2022-11-09 08:06:50,285] Trial 3 finished with value: 0.8823137665303015 and parameters: {'C': 54.80385100006619}. Best is trial 0 with value: 0.8823137665303015.
[I 2022-11-09 08:06:51,848] Trial 4 finished with value: 0.8823137665303015 and parameters: {'C': 79.50619999913107}. Best is trial 0 with value: 0.8823137665303015.
[I 2022-11-09 08:06:53,624] Trial 5 finished with valu

Best trial:
  F1-Value:  0.8823137665303015
  Params: 
.   C: 78.11692480616955


In [ ]:
# save model
file_name = "../Models/lg.pkl"
lg_model = LogisticRegression(**trial.params) 
lg_model.fit(X_train, y_train)
pickle.dump(lg_model, open(file_name, "wb"))

# load
lg_model_loaded = pickle.load(open(file_name, "rb"))

3g. Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
def objective(trial):

    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 200),
        'max_depth': trial.suggest_int('max_depth', 5, 50),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 2, 20)
    }

    model = RandomForestClassifier(**params)
    
    model.fit(X_train, y_train)
  
    prediction = model.predict(X_test)
    f1 = f1_score(y_test, prediction)
    return f1

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=25)         # change n_trials as see fit

print("Best trial:")
trial = study.best_trial
print("  F1-Value: ",  trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print(".   {}: {}".format(key, value))

[I 2022-11-09 08:07:40,711] A new study created in memory with name: no-name-20096c79-2fed-4c08-a48f-bcd7597b794e
[I 2022-11-09 08:09:40,039] Trial 0 finished with value: 0.8714751310755279 and parameters: {'n_estimators': 140, 'max_depth': 41, 'min_samples_split': 9, 'min_samples_leaf': 4}. Best is trial 0 with value: 0.8714751310755279.
[I 2022-11-09 08:12:22,689] Trial 1 finished with value: 0.8729971167934962 and parameters: {'n_estimators': 198, 'max_depth': 20, 'min_samples_split': 8, 'min_samples_leaf': 3}. Best is trial 1 with value: 0.8729971167934962.
[I 2022-11-09 08:13:06,299] Trial 2 finished with value: 0.8689799014031097 and parameters: {'n_estimators': 55, 'max_depth': 37, 'min_samples_split': 16, 'min_samples_leaf': 5}. Best is trial 1 with value: 0.8729971167934962.
[I 2022-11-09 08:15:08,389] Trial 3 finished with value: 0.8683250414593698 and parameters: {'n_estimators': 173, 'max_depth': 35, 'min_samples_split': 6, 'min_samples_leaf': 15}. Best is trial 1 with valu

Best trial:
  F1-Value:  0.8730211237654174
  Params: 
.   n_estimators: 158
.   max_depth: 32
.   min_samples_split: 7
.   min_samples_leaf: 2


In [ ]:
# save model
file_name = "../Models/rnd.pkl"
rnd_model = RandomForestClassifier(**trial.params) 
rnd_model.fit(X_train, y_train)
pickle.dump(rnd_model, open(file_name, "wb"))

# load
rnd_model_loaded = pickle.load(open(file_name, "rb"))

# 4. Model Evaluation Results

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
from keras.models import save_model, load_model

model_results = pd.DataFrame()

# Models that only use embedding as X values
pure_text_models = {
    'LSTM': load_model('../Models/LSTM.h5'),
    'XGBoost': pickle.load(open('../Models/xgb.pkl', "rb")),
    'LightGBM': pickle.load(open('../Models/lgb.pkl', "rb")),
    'SVM': pickle.load(open('../Models/svm.pkl', "rb")),
    'Logistic Reg': pickle.load(open('../Models/lg.pkl', "rb")),
    'Random Forest': pickle.load(open('../Models/rnd.pkl', "rb")),
    'CNN': load_model('../Models/CNN.h5'),
}

for model_name, model in pure_text_models.items():
  y_pred = model.predict(X_test)
  try:
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
  except ValueError as e:
    y_pred_class = list(map(lambda x: 1 if x > 0.5 else 0, y_pred))
    accuracy = accuracy_score(y_test, y_pred_class)
    f1 = f1_score(y_test, y_pred_class, average='weighted')
    precision = precision_score(y_test, y_pred_class)
    recall = recall_score(y_test, y_pred_class)
  model_results = model_results.append(
      {
       'Model': model_name, 
       'Accuracy': accuracy, 
       'F1-score': f1,
       'Precision': precision,
       'Recall': recall,
      #  'ROC AUC': roc_auc
       }, ignore_index = True)


862/862 [==============================] - 3s 3ms/step


In [ ]:
pd.DataFrame(model_results).to_csv('../Model_Evaluation.csv', index=False)

In [ ]:
model_results

,Model,Accuracy,F1-score,Precision,Recall
0,LSTM,0.902220,0.902100,0.880611,0.868645
1,XGBoost,0.912271,0.912105,0.896376,0.877981
2,LightGBM,0.908388,0.908270,0.888795,0.876225
3,SVM,0.910602,0.910176,0.906007,0.861620
4,Logistic Reg,0.910239,0.909736,0.908859,0.857275
5,Random Forest,0.901930,0.901466,0.894277,0.850712
6,CNN,0.913432,0.913176,0.902367,0.874006


In [ ]:
# with BERT
pd.read_csv('../Model_Evaluation.csv')

,Model,Accuracy,F1-score,Precision,Recall
0,LSTM,0.902220,0.902100,0.880611,0.868645
1,XGBoost,0.912271,0.912105,0.896376,0.877981
2,LightGBM,0.908388,0.908270,0.888795,0.876225
3,SVM,0.910602,0.910176,0.906007,0.861620
4,Logistic Reg,0.910239,0.909736,0.908859,0.857275
5,Random Forest,0.901930,0.901466,0.894277,0.850712
6,CNN,0.913432,0.913176,0.902367,0.874006
